<div class="alert alert-block alert-info">

# Imports

</div>

In [1]:
!pip install talos

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import talos as ta
from talos.utils import hidden_layers, early_stopper

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, RMSprop
from keras.activations import relu, elu
from keras.losses import binary_crossentropy

<div class="alert alert-block alert-info">

# Dataframes

</div>

In [0]:
df_features = pd.read_csv('ted_main_features.csv',sep="|",quotechar='"')

In [4]:
df_features.columns

Index(['comments', 'duration', 'languages', 'num_speaker', 'views',
       'film_month', 'film_dayofweek', 'published_month',
       'published_dayofweek', 'event_TED',
       ...
       'world', 'worldwide', 'worth', 'write', 'writer', 'wrong', 'ye', 'year',
       'york', 'young'],
      dtype='object', length=580)

In [5]:
df_features.shape

(2550, 580)

In [6]:
df_features.head()

,comments,duration,languages,num_speaker,views,film_month,film_dayofweek,published_month,published_dayofweek,event_TED,event_TEDx,event_noTED,previous_talks,previous_talk_views,previous_views_sum,previous_views_max,previous_views_min,rating_funny,rating_beautiful,rating_ingenious,rating_courageous,rating_longwinded,rating_confusing,rating_informative,rating_fascinating,rating_unconvincing,rating_persuasive,rating_jaw-dropping,rating_ok,rating_obnoxious,rating_inspiring,tag_technology,tag_science,tag_design,tag_business,tag_collaboration,tag_innovation,tag_social_change,tag_health,tag_nature,...,turn,understand,unexpected,unique,universe,urge,use,using,value,video,view,violence,vision,visual,voice,wa,walk,want,war,watch,water,way,web,win,wisdom,wish,woman,wonder,word,work,world,worldwide,worth,write,writer,wrong,ye,year,york,young
0,10,810,1,1,418368,8,5,9,1,1,0,0,0,0,0,0,0,0,9,4,8,1,0,37,15,1,30,0,6,1,31,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3,795,1,1,542088,8,5,9,4,1,0,0,0,0,0,0,0,6,41,7,43,3,1,15,14,5,2,3,1,5,35,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,8,723,4,1,707788,8,5,9,4,1,0,0,0,0,0,0,0,19,54,12,77,6,1,71,34,2,63,16,3,1,72,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,10,934,2,1,527314,8,5,9,2,1,0,0,0,0,0,0,0,2,7,19,6,1,0,78,28,3,36,3,0,0,73,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,33,722,4,1,613915,7,2,9,3,1,0,0,0,0,0,0,0,4,15,3,2,11,4,157,35,38,62,7,9,18,18,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


<div class="alert alert-block alert-info">

# Dummies

</div>

## Create Dummies

In [0]:
df_film_month = pd.get_dummies(df_features['film_month'],prefix='film_month',drop_first=True)
df_published_month = pd.get_dummies(df_features['published_month'],prefix='published_month',drop_first=True)

df_film_day = pd.get_dummies(df_features['film_dayofweek'],prefix='film_dayofweek',drop_first=True)
df_published_day = pd.get_dummies(df_features['published_dayofweek'],prefix='published_dayofweek',drop_first=True)

In [8]:
df_film_month.head(2)

,film_month_2,film_month_3,film_month_4,film_month_5,film_month_6,film_month_7,film_month_8,film_month_9,film_month_10,film_month_11,film_month_12
0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0


In [9]:
df_film_day.head(2)

,film_dayofweek_1,film_dayofweek_2,film_dayofweek_3,film_dayofweek_4,film_dayofweek_5,film_dayofweek_6
0,0,0,0,0,1,0
1,0,0,0,0,1,0


In [10]:
print(df_film_month.shape)
print(df_film_day.shape)

(2550, 11)
(2550, 6)


In [11]:
df_published_month.head(2)

,published_month_2,published_month_3,published_month_4,published_month_5,published_month_6,published_month_7,published_month_8,published_month_9,published_month_10,published_month_11,published_month_12
0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0


In [12]:
df_published_day.head(2)

,published_dayofweek_1,published_dayofweek_2,published_dayofweek_3,published_dayofweek_4,published_dayofweek_5,published_dayofweek_6
0,1,0,0,0,0,0
1,0,0,0,1,0,0


In [13]:
print(df_published_month.shape)
print(df_published_day.shape)

(2550, 11)
(2550, 6)


## Drop Columns

In [14]:
df_features = df_features.drop(columns=['film_month','film_dayofweek','published_month','published_dayofweek'])
df_features.head()

,comments,duration,languages,num_speaker,views,event_TED,event_TEDx,event_noTED,previous_talks,previous_talk_views,previous_views_sum,previous_views_max,previous_views_min,rating_funny,rating_beautiful,rating_ingenious,rating_courageous,rating_longwinded,rating_confusing,rating_informative,rating_fascinating,rating_unconvincing,rating_persuasive,rating_jaw-dropping,rating_ok,rating_obnoxious,rating_inspiring,tag_technology,tag_science,tag_design,tag_business,tag_collaboration,tag_innovation,tag_social_change,tag_health,tag_nature,tag_environment,tag_future,tag_communication,tag_activism,...,turn,understand,unexpected,unique,universe,urge,use,using,value,video,view,violence,vision,visual,voice,wa,walk,want,war,watch,water,way,web,win,wisdom,wish,woman,wonder,word,work,world,worldwide,worth,write,writer,wrong,ye,year,york,young
0,10,810,1,1,418368,1,0,0,0,0,0,0,0,0,9,4,8,1,0,37,15,1,30,0,6,1,31,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3,795,1,1,542088,1,0,0,0,0,0,0,0,6,41,7,43,3,1,15,14,5,2,3,1,5,35,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,8,723,4,1,707788,1,0,0,0,0,0,0,0,19,54,12,77,6,1,71,34,2,63,16,3,1,72,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,10,934,2,1,527314,1,0,0,0,0,0,0,0,2,7,19,6,1,0,78,28,3,36,3,0,0,73,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,33,722,4,1,613915,1,0,0,0,0,0,0,0,4,15,3,2,11,4,157,35,38,62,7,9,18,18,0,0,0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


## Join Dummies

In [15]:
df_features = df_features.merge(df_film_month, left_index=True, right_index=True)
df_features = df_features.merge(df_published_month, left_index=True, right_index=True)
df_features = df_features.merge(df_film_day, left_index=True, right_index=True)
df_features = df_features.merge(df_published_day, left_index=True, right_index=True)
df_features.head()

,comments,duration,languages,num_speaker,views,event_TED,event_TEDx,event_noTED,previous_talks,previous_talk_views,previous_views_sum,previous_views_max,previous_views_min,rating_funny,rating_beautiful,rating_ingenious,rating_courageous,rating_longwinded,rating_confusing,rating_informative,rating_fascinating,rating_unconvincing,rating_persuasive,rating_jaw-dropping,rating_ok,rating_obnoxious,rating_inspiring,tag_technology,tag_science,tag_design,tag_business,tag_collaboration,tag_innovation,tag_social_change,tag_health,tag_nature,tag_environment,tag_future,tag_communication,tag_activism,...,writer,wrong,ye,year,york,young,film_month_2,film_month_3,film_month_4,film_month_5,film_month_6,film_month_7,film_month_8,film_month_9,film_month_10,film_month_11,film_month_12,published_month_2,published_month_3,published_month_4,published_month_5,published_month_6,published_month_7,published_month_8,published_month_9,published_month_10,published_month_11,published_month_12,film_dayofweek_1,film_dayofweek_2,film_dayofweek_3,film_dayofweek_4,film_dayofweek_5,film_dayofweek_6,published_dayofweek_1,published_dayofweek_2,published_dayofweek_3,published_dayofweek_4,published_dayofweek_5,published_dayofweek_6
0,10,810,1,1,418368,1,0,0,0,0,0,0,0,0,9,4,8,1,0,37,15,1,30,0,6,1,31,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
1,3,795,1,1,542088,1,0,0,0,0,0,0,0,6,41,7,43,3,1,15,14,5,2,3,1,5,35,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
2,8,723,4,1,707788,1,0,0,0,0,0,0,0,19,54,12,77,6,1,71,34,2,63,16,3,1,72,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
3,10,934,2,1,527314,1,0,0,0,0,0,0,0,2,7,19,6,1,0,78,28,3,36,3,0,0,73,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
4,33,722,4,1,613915,1,0,0,0,0,0,0,0,4,15,3,2,11,4,157,35,38,62,7,9,18,18,0,0,0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0


<div class="alert alert-block alert-info">

# Train, Test, Split

</div>

In [0]:
# Generamos la matriz X y el vector y
X = df_features.drop(columns=['views'])
y = df_features['views']

In [0]:
# Separamos train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [20]:
# Estandarizamos las variables
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Chequeamos las shapes
print('X train shape: ', X_train.shape)
print('X test shape: ', X_test.shape)
print('y train shape: ', y_train.shape)
print('y test shape: ', y_test.shape)

X train shape:  (1785, 609)
X test shape:  (765, 609)
y train shape:  (1785,)
y test shape:  (765,)


In [0]:
# Primero, definimos una función que permita construir el modelo, donde todos los valores de los hiperparámetros se obtienen del diccionario params 

def views_ted_talks_model(x_train, y_train, x_val, y_val, params):

    model = Sequential()
    
    model.add(Dense(params['first_neuron'], input_dim=x_train.shape[1],
                    activation=params['activation']))
    
    hidden_layers(model, params, 1) # Necesario para explorar distintas configuraciones de capas ocultas
    
    model.add(Dense(1, activation=params['last_activation']))
    
    model.compile(optimizer=params['optimizer'],
                  loss='binary_crossentropy',
                  metrics=['mae'])
    
    history = model.fit(x_train, y_train,
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=0,
                        callbacks=[early_stopper(params['epochs'])])

    return history, model

In [0]:
# Segundo, almacenamos en un diccionario el espacio de hiperparámetros a explorar

p = {'first_neuron':[128, 256, 64, 32],     # Requerido si queremos probar distinta cantidad de hidden_layers
     'hidden_layers':[2, 3, 4, 5, 6, 7, 8],     # Requerido si queremos probar distinta cantidad de hidden_layers
     'dropout': [0, 0.1, 0.3, 0.5, 0.6],   # Requerido si queremos probar distinta cantidad de hidden_layers
     'shapes': ['brick','triangle'],              # Requerido si queremos probar distinta cantidad de hidden_layers 
     'batch_size': [512, 256, 128, 64, 32, 16],
     'epochs': [10, 20, 50, 100, 300],
     'optimizer': [Adam,RMSprop,Adam(lr=0.003),RMSprop(lr=0.003)],
     'activation':[relu,elu],
     'last_activation': [relu,elu]}